# 3. Cassandra to Arrow

We use some code from the Cassandra server to read the SSTable, but instead of de/serializing to/from CQL, we use an [Arrow IPC stream](http://arrow.apache.org/), which is stored in a columnar format and better suited for analytics.

Data transformations:

1. SSTable on disk
2. Deserialized into Java Object in C* server
3. Client makes request to server (not to C* DB)
4. Data serialized via Arrow IPC stream
5. Sent across network
6. Arrow IPC stream received by client
7. Transformed into Arrow Table / cuDF

**Pros:**
- doesn't make request to the main Cassandra DB, which lessens the load and allows for other operations to run
- less de/serialization involved using the Arrow IPC stream

**Cons:**
- don't want to have to start Cassandra or use the JVM
- complex architecture

In [1]:
import pyarrow as pa
import pandas as pd
import socket

HOST = '127.0.0.1'
PORT = 9143

In [2]:
def read_bytes(sock, n):
    data = b''
    while len(data) < n:
        more = sock.recv(n - len(data))
        if not more:
            raise EOFError("Socket connection ended before reading specified number of bytes")
        data += more
    return data

def read_u8(sock):
    data = read_bytes(sock, 8)
    return int.from_bytes(data, byteorder='big')

# read data from socket
def fetch_data():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((HOST, PORT))
        sock.sendall(b'hello world\n')
        num_tables = read_u8(sock)
        table_buffers = []
        for i in range(num_tables):
            print('receiving table', i)
            table_size = read_u8(sock)
            buf = read_bytes(sock, table_size)
            table_buffers.append(buf)
    return table_buffers

In [23]:
buffers = fetch_data()
tables = [pa.ipc.open_stream(buf).read_all() for buf in buffers]
len(tables)

receiving table 0


1

In [24]:
tables[0].to_pandas()

,partition key,_ts__row,value,_ts__value
0,6,2021-07-15 16:51:35.440203,439790106,NaT
1,7,2021-07-15 16:51:35.442510,564330072,NaT
2,9,2021-07-15 16:51:35.446255,97405552,NaT
3,4,2021-07-15 16:51:35.434989,351686621,NaT
4,3,2021-07-15 16:51:35.416599,352527683,NaT
5,5,2021-07-15 16:51:35.437366,114304900,NaT
6,0,2021-07-15 16:51:35.403305,382062539,NaT
7,8,2021-07-15 16:51:35.444276,296173906,NaT
8,2,2021-07-15 16:51:35.414324,949364593,NaT
9,1,2021-07-15 16:51:35.409490,774912474,NaT


In [18]:
tables[1].to_pandas()

IndexError: list index out of range

In [12]:
tables[2].to_pandas()

,partition key,_ts__row,count,_ts__count,size,_ts__size,types,_ts__types
0,bar,2021-07-20 19:45:23.434589,456.0,NaT,None,NaT,"[cyan, red, yellow]","[NaT, NaT, NaT]"
1,foo,2021-07-20 19:45:00.539860,123.0,NaT,small,NaT,None,None
2,emptyrow,2021-07-20 19:45:39.919177,NaN,NaT,None,NaT,None,None
